# Постановка задачи
Рассматривается задача из
http://www.mathnet.ru/links/18bdadf51cc40c6f125f36b7a947bd09/at1291.pdf

В качестве целевой функции возьмем выпуклую кусочно-линейную (полиэдральную функцию)
$$
\Phi(u, X) = \max\limits_{i=1,\ldots,n}\{A_{1i}^Tu + B_{1i}^TX + b_{1i}\}
$$
Ограничения задаются функцией
$$
Q(u, X) = \max\limits_{i=1,\ldots,m}\{A_{2i}^Tu + B_{2i}^TX + b_{2i}\}
$$

$A_{1i}^T, B_{1i}^T, A_{2i}^T, B_{2i}^T$ - строки неслучайных матриц

Будем максимизировать вероятность
$$
P_{\varphi}(u) \triangleq P\{\Phi(u, X)\le\varphi, Q(u, X)\le 0\}
$$
Множество допустимых управлений - выпуклый многогранник
$$
U \triangleq \{u\in R^n: A_3u\le b_3, u\ge 0\}
$$

## Пример
\begin{equation}
A_1 = \begin{pmatrix}
6 & -6\\
1 & 1
\end{pmatrix},
B_1 = \begin{pmatrix}
1 & 2\\
10 & -10
\end{pmatrix},
b_1 = \begin{pmatrix}
0\\
-5
\end{pmatrix}
\end{equation}

\begin{equation}
A_2 = \begin{pmatrix}
10 & -10
\end{pmatrix},
B_2 = \begin{pmatrix}
-3 & -1
\end{pmatrix},
b_2 = \begin{pmatrix}
-2
\end{pmatrix}
\end{equation}
Множество допустимых управлений
$$
U = \{u: u\in R^2, 0\le u_1\le 10, 0\le u_2, 10\}
$$

В работе по ссылке выше переходили к задаче минимизации функции квантили. Оптимальное значение квантили получилось 13.906, его и возьмем в качестве $\varphi$

Параметры задачи также можно взять из
http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=at&paperid=3598&option_lang=rus
Там размерность повыше, но распределение дискретное.

Код далее рассчитан на произвольные размерности матриц, все собирается автоматом.

In [14]:
from LFlow.stochastic_model import IndependentGenerator, MultivariateGaussGenerator, DiscreteVectorGenerator, stats
from LFlow.labos_flow_v2 import LabFunc, np, plt, LabSigmoid, Identity, LabSmoothmax, LabIndicator, LabMax
from LFlow.labos_point import Point
np.seterr(all='ignore')

alpha1 = 0.9
alpha2 = 0.8
steps = 100
tol = 1e-4
"""
шаг считаем динамически таким образом,
чтобы максимальное изменение одной из координат вектора u
было равно step. При ухудшении критерия,
шаг уменьшается в 2 раза
"""
step = 0.1

A1 = np.array([[6, -6],
               [1, 1]])
B1 = np.array([[1, 2],
               [10, -10]])
b1 = np.array([0, -5])
A2 = np.array([[10, -10]])
B2 = np.array([[-3, -1]])
b2 = np.array([-2])
ulims = np.array([[0, 0],
                  [10, 10]])

"""
генерация выборки из многомерного стандартного гаусса
"""
sm = MultivariateGaussGenerator(['X1', 'X2'])
_, sample = sm.rvs(3000)

base_point = Point({'u0x1' : 0.65, 'u0x2' : 1.3, 'phi' : 13.906})
# quasi-gradient solution
opt_point = Point({'u0x1' : -0.0025, 'u0x2' : 1.337, 'phi' : 13.906})

In [15]:
"""
задаем переменные и функции
"""
phi = Identity('phi')
phis = []
args = []
u_args = []
x_args = []
for i in range(A1.shape[1]):
    args.append('u{}'.format(str(hex(i+1))))
    u_args.append('u{}'.format(str(hex(i+1))))
for i in range(B1.shape[1]):
    args.append(f'X{i+1}')
    x_args.append(f'X{i+1}')
for i in range(A1.shape[0]):
    # набиваем линейные функции из критерия
    cmd = '{}*u{}'.format(A1[i, 0], str(hex(1)))
    derivs = {'u0x1' : A1[i, 0]}
    for j in range(1, A1.shape[1]):
        if A1[i, j] < 0:
            cmd += ' - {}*u{}'.format(np.abs(A1[i, j]), str(hex(j+1)))
        elif A1[i, j] > 0:
            cmd += ' + {}*u{}'.format(np.abs(A1[i, j]), str(hex(j+1)))
        derivs['u{}'.format(str(hex(j+1)))] = A1[i, j]
    
    for j in range(0, B1.shape[1]):
        if B1[i, j] < 0:
            cmd += ' - {}*X{}'.format(np.abs(B1[i, j]), j+1)
        elif B1[i, j] > 0:
            cmd += ' + {}*X{}'.format(np.abs(B1[i, j]), j+1)
        derivs[f'X{j+1}'] = B1[i, j]
        
    if b1[i] < 0:
        cmd += ' - {}'.format(np.abs(b1[i]))
    elif b1[i] > 0:
        cmd += ' + {}'.format(np.abs(b1[i]))
    phis.append(LabFunc(cmd, derivs, args=args))


qs = []
for i in range(A2.shape[0]):
    # набиваем линейные функции из критерия
    cmd = '{}*u{}'.format(A2[i, 0], str(hex(1)))
    derivs = {'u0x1' : A2[i, 0]}
    for j in range(1, A2.shape[1]):
        if A2[i, j] < 0:
            cmd += ' - {}*u{}'.format(np.abs(A2[i, j]), str(hex(j+1)))
        elif A2[i, j] > 0:
            cmd += ' + {}*u{}'.format(np.abs(A2[i, j]), str(hex(j+1)))
        derivs['u{}'.format(str(hex(j+1)))] = A2[i, j]
    
    for j in range(0, B2.shape[1]):
        if B2[i, j] < 0:
            cmd += ' - {}*X{}'.format(np.abs(B2[i, j]), j+1)
        elif B2[i, j] > 0:
            cmd += ' + {}*X{}'.format(np.abs(B2[i, j]), j+1)
        derivs[f'X{j+1}'] = B2[i, j]
        
    if b2[i] < 0:
        cmd += ' - {}'.format(np.abs(b2[i]))
    elif b2[i] > 0:
        cmd += ' + {}'.format(np.abs(b2[i]))
    qs.append(LabFunc(cmd, derivs, args=args))
 


theta = 12
phis_max = LabSmoothmax(*phis, theta=theta)
cond1 = LabSigmoid(phi - phis_max, theta=theta)
if len(qs) > 1:
    cond2 = LabSigmoid( - LabSmoothmax(*qs, theta=theta), theta=theta)
else:
    cond2 = LabSigmoid( - qs[0], theta=theta)
straight_cond1 = LabIndicator(phi - LabMax(*phis))
straight_cond2 = LabIndicator(-LabMax(*qs))
pphi = cond1*cond2
straight_pphi = straight_cond1*straight_cond2

In [16]:
for i in range(steps):
    p, p_grad = sm.papa_carlo(pphi, base_point, sample, derivs=u_args)
    cur_step = step/max([abs(pg) for pg in p_grad.dict.values()])
    for j, uu in enumerate(u_args):
        """
        если уперлись в ограничение, то не двигаемся в этом направлении
        """
        if base_point[uu] <= ulims[0, j] + tol:
            p_grad[uu] = max(0, p_grad[uu])
        if base_point[uu] >= ulims[1, j] - tol:
            p_grad[uu] = min(0, p_grad[uu])
        if (base_point[uu] + cur_step*p_grad[uu] < ulims[0, j]):
            cur_step = min(cur_step, (ulims[0, j] - base_point[uu])/p_grad[uu])
        if (base_point[uu] + cur_step*p_grad[uu] > ulims[1, j]):
            cur_step = min(cur_step, (ulims[1, j] - base_point[uu])/p_grad[uu])
    candidate_point = Point(base_point.dict.copy())  
    candidate_point += cur_step*p_grad
    candidate_p = sm.papa_carlo(pphi, candidate_point, sample)
    if candidate_p <= p + 0.01*tol:
        step = 0.7*step
    else:
        base_point += cur_step*p_grad
        print(base_point.shrink(u_args))

{'u0x1': 0.55, 'u0x2': 1.3513958532789756}
{'u0x1': 0.45000000000000007, 'u0x2': 1.3525779975229604}
{'u0x1': 0.3500000000000001, 'u0x2': 1.2525858187607242}
{'u0x1': 0.2500000000000001, 'u0x2': 1.1525917285800917}
{'u0x1': 0.1500000000000001, 'u0x2': 1.0525962803217253}
{'u0x1': 0.050000000000000114, 'u0x2': 0.952600826509127}
{'u0x1': 0.0, 'u0x2': 0.9026040701403235}
{'u0x1': 0.0, 'u0x2': 0.8026106769161367}
{'u0x1': 0.0, 'u0x2': 0.7930346868679148}
{'u0x1': 0.0, 'u0x2': 0.7749283444422421}


In [13]:
print(p)

0.9085730980957372
